In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
sync_filt = Filter.create_filter(address=None, event_types=[Pair.events.Sync])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=sync_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])
    pair: PairDetails = FetchPairDetails().apply(web3, evt['address'])

    amt0 = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    amt1 = Conversion().convert_int256_bytes_to_int(arguments[1])  
    amt0_human = amt0/(10**pair.token0.decimals)
    amt1_human = amt1/(10**pair.token1.decimals)    

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt['event']
    event['details']['token0'] = pair.token0.address
    event['details']['token1'] = pair.token1.address
    event['details']['token0_symbol'] = pair.token0.symbol
    event['details']['token1_symbol'] = pair.token1.symbol 
    event['details']['token0_decimal'] = pair.token0.decimals
    event['details']['token1_decimal'] = pair.token1.decimals        
    event['details']['amount0'] = amt0
    event['details']['amount1'] = amt1
    event['details']['price'] = amt1_human/amt0_human
    
    events[k] = event
    if key not in processed_events:
        print(f"Sync at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Sync at block:60,958,062 tx:0x518408e20b70cb73148635ffd9ea1d6664ac15704264cff80170815cc62b4b4c
Sync at block:60,958,062 tx:0x93d0661837da696c6699b8c198a7aae82636fb6caab7adb80d9436c7e0e35e39
Sync at block:60,958,062 tx:0x93d0661837da696c6699b8c198a7aae82636fb6caab7adb80d9436c7e0e35e39
Sync at block:60,958,063 tx:0x96c5570f69519f9660497965b657b75031d80e2006d3bf9b3c9f78f21aac290e
Sync at block:60,958,064 tx:0xbf7198726ccb619171e3390bd11a3727af98600ebdd0083c3b231114d04ba2c3
Sync at block:60,958,065 tx:0xc49b310864dc736893c103bb142edb79bf94db24c735334b5908f6c5c7d1ad98
Sync at block:60,958,068 tx:0xd086e433e6ba809c9a5fcd864c9469df2ecbf5cd9a01a1a1953f8cda82ce0da9
Sync at block:60,958,068 tx:0xd086e433e6ba809c9a5fcd864c9469df2ecbf5cd9a01a1a1953f8cda82ce0da9
Sync at block:60,958,068 tx:0xd086e433e6ba809c9a5fcd864c9469df2ecbf5cd9a01a1a1953f8cda82ce0da9
Sync at block:60,958,068 tx:0x5184adc0695cf9aa17e8bcc5b1fc436762bfbdd83a6793b74532cd8856e2095f
Sync at block:60,958,069 tx:0xb012073d4cd8020bff3b

In [8]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,sync,sushi,0xadbf1854e5883eb8aa7baf50705338739e558e5b,0x518408e20b70cb73148635ffd9ea1d6664ac15704264...,60958062,1724452109,{'web3_type': <class 'web3._utils.datatypes.Sy...
1,polygon,uniswapv2pair,sync,sushi,0xd0fa2eaa5d854f184394e93f7b75624084600685,0x93d0661837da696c6699b8c198a7aae82636fb6caab7...,60958062,1724452109,{'web3_type': <class 'web3._utils.datatypes.Sy...
2,polygon,uniswapv2pair,sync,sushi,0x1534d7c91bd77eb447acb7fb92ea042b918f58bb,0x93d0661837da696c6699b8c198a7aae82636fb6caab7...,60958062,1724452109,{'web3_type': <class 'web3._utils.datatypes.Sy...
3,polygon,uniswapv2pair,sync,sushi,0x52d52b2592001537e2a7f973eac2a9fc640e6ccd,0x96c5570f69519f9660497965b657b75031d80e2006d3...,60958063,1724452111,{'web3_type': <class 'web3._utils.datatypes.Sy...
4,polygon,uniswapv2pair,sync,sushi,0xc3379226aeef21464d05676305dad1261d6f3fac,0xbf7198726ccb619171e3390bd11a3727af98600ebdd0...,60958064,1724452115,{'web3_type': <class 'web3._utils.datatypes.Sy...
5,polygon,uniswapv2pair,sync,sushi,0xb9b36bddc5f4c07e46d697e4b126417fd8412876,0xc49b310864dc736893c103bb142edb79bf94db24c735...,60958065,1724452117,{'web3_type': <class 'web3._utils.datatypes.Sy...
6,polygon,uniswapv2pair,sync,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xd086e433e6ba809c9a5fcd864c9469df2ecbf5cd9a01...,60958068,1724452123,{'web3_type': <class 'web3._utils.datatypes.Sy...
7,polygon,uniswapv2pair,sync,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xd086e433e6ba809c9a5fcd864c9469df2ecbf5cd9a01...,60958068,1724452123,{'web3_type': <class 'web3._utils.datatypes.Sy...
8,polygon,uniswapv2pair,sync,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xd086e433e6ba809c9a5fcd864c9469df2ecbf5cd9a01...,60958068,1724452123,{'web3_type': <class 'web3._utils.datatypes.Sy...
9,polygon,uniswapv2pair,sync,sushi,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,0x5184adc0695cf9aa17e8bcc5b1fc436762bfbdd83a67...,60958068,1724452123,{'web3_type': <class 'web3._utils.datatypes.Sy...


In [9]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0xadbf1854e5883eb8aa7baf50705338739e558e5b',
  'tx_hash': '0x518408e20b70cb73148635ffd9ea1d6664ac15704264cff80170815cc62b4b4c',
  'blk_num': 60958062,
  'timestamp': 1724452109,
  'details': {'web3_type': web3._utils.datatypes.Sync,
   'token0': '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270',
   'token1': '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619',
   'token0_symbol': 'WMATIC',
   'token1_symbol': 'WETH',
   'token0_decimal': 18,
   'token1_decimal': 18,
   'amount0': 1273080220259469227424437,
   'amount1': 246666738305082383417,
   'price': 0.0001937558485158215}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0xd0fa2eaa5d854f184394e93f7b75624084600685',
  'tx_hash': '0x93d0661837da696c6699b8c198a7aae82636fb6caab7adb80d9436c7e0e35e39',
  'blk_num': 60958062,
  'timestamp': 1724452109,
  'details': {'web3_type':